In [11]:
import pandas as pd
from custom_functions_scRBA import *

In [12]:
#### LOAD INPUTS AND PARAMETERS
# Stoichiometry
df_stoich = pd.read_excel('./model/RBA_stoichiometry.xlsx')
df_stoich.index = df_stoich.id.to_list()

# Enzymes' apparent turnover rates (k_app) (unit: 1/s)
# (script will convert the unit to 1/h)
df_kapp = pd.read_excel('./input/PARAMS_kapp_compiled.xlsx')
df_kapp.index = df_kapp.id.to_list()

# Load protein sequence lengths
df_pro = pd.read_excel('./input/PROTEIN_stoich_curation.xlsx')
df_pro.index = df_pro.id.to_list()

# Ribosome efficiency (amino acids translated per active ribosome per second)
# (script will convert the unit to per hour)
kribo = 10.5

# Dummy protein's length (assigned to be the median protein length)
NAA_dummy = 401

In [13]:
eqn_list = []; sl_list = [];

In [14]:
### Ribosome synthesis requirement coupled to protein translation
# Mitochondrial
idx = df_stoich[(df_stoich.coupling_type == 'prot_ribo') & \
                (df_stoich.coupling_species == 'ribomito')].index
lhs = "(v('RIBOSYN-ribomito')"
lhs += " + (slU('RIBOSYN-ribomito') - slL('RIBOSYN-ribomito')) / MW('RIBOSYN-ribomito')) * %kribomito%"
rhs = "%mu% * sum(j$mito_translation(j), NAA(j) * v(j))"
eqn_list.append(lhs + ' =e= ' + rhs + ';')

sl_list.append('RIBOSYN-ribomito')
rxn_list = ["'" + i + "'" for i in idx if i != '']
rxn_list = ['/'] + rxn_list + ['/']
#with open('./model/RBA_mito_translation.txt', 'w') as f:
#    f.write('\n'.join(rxn_list))

In [15]:
# Nucleus
idx = df_stoich[(df_stoich.coupling_type == 'prot_ribo') & \
                (df_stoich.coupling_species == 'ribonuc')].index
lhs = "(v('RIBOSYN-ribonuc')"
lhs += " + (slU('RIBOSYN-ribonuc') - slL('RIBOSYN-ribonuc')) / MW('RIBOSYN-ribonuc'))* %kribonuc%"
rhs = "%mu% * sum(j$nuc_translation(j), NAA(j) * v(j))"
eqn_list.append(lhs + ' =e= ' + rhs + ';')

sl_list.append('RIBOSYN-ribonuc')
rxn_list = ["'" + i + "'" for i in idx if i != '']
rxn_list = ['/'] + rxn_list + ['/']
#with open('./model/RBA_nuc_translation.txt', 'w') as f:
#    f.write('\n'.join(rxn_list))

In [16]:
### Enzyme synthesis requirement coupled to metabolic reaction rate
idx = df_stoich[df_stoich.coupling_type == 'rxn_enz'].index
for i in idx:
    eid = df_stoich.id[i][4:]
    kapp = round(df_kapp.loc[i, 'kapp (1/s)']*3600, 6)
    lhs = "(v('ENZSYN-" + eid + "') " + \
        "+ (slU('ENZSYN-" + eid + "') - slL('ENZSYN-" + eid + "')) / MW('ENZSYN-" + eid + "'))" + \
        " * " + str(kapp)
    rhs = "%mu% * v('" + i + "') "
    
    eqn_list.append(lhs + ' =e= ' + rhs + ';')
    sl_list.append('ENZSYN-' + eid)
    
eqn_idx = ['eqn'+str(i) for i in range(0, len(eqn_list))]
eqn_list = ['eqn'+str(i)+'.. ' + eqn_list[i] for i in range(0, len(eqn_list))]

with open('./model/RBA_capacityConstraints_slacked_declares.txt', 'w') as f:
    f.write('\n'.join(eqn_idx))
with open('./model/RBA_capacityConstraints_slacked_eqns.txt', 'w') as f:
    f.write('\n'.join(eqn_list))

In [17]:
### List of slack variables
sl_list = ["'" + i + "'" for i in sl_list if i != '']
sl_list = ['/'] + sl_list + ['/']
with open('./model/RBA_rxns_slack_apply.txt', 'w') as f:
    f.write('\n'.join(sl_list))

In [18]:
df_enz = pd.read_excel('./input/ENZYME_stoich_curation.xlsx')
df_enz.index = df_enz.id.to_list()

In [19]:
### List of species MW for slack variable scaling
mw_list = []
mw_list.append("'RIBOSYN-ribonuc' 3336.65686")
mw_list.append("'RIBOSYN-ribomito' 3647.27370")
for i in sl_list[3:-1]:
    eid = i[1:-1]
    eid_conv = 'RXN-' + eid[7:]
    mw_list.append("'" + eid + "' " + str(df_enz.loc[eid_conv, 'MW (g/mmol)']))
    
mw_list = ['/'] + mw_list + ['/']
with open('./model/RBA_slack_species_MW.txt', 'w') as f:
    f.write('\n'.join(mw_list))

In [20]:
mw_list

['/',
 "'RIBOSYN-ribonuc' 3336.65686",
 "'RIBOSYN-ribomito' 3647.27370",
 "'ENZSYN-BTDDH_c_FWD-YAL060W' 91.1741",
 "'ENZSYN-BTDDH_c_REV-YAL060W' 91.1741",
 "'ENZSYN-13GS_c_FWD-GSC2RHO1' 238.33867",
 "'ENZSYN-13GS_c_FWD-GSC1RHO1' 241.31742",
 "'ENZSYN-16GS_c_FWD-YPR159W' 81.57498",
 "'ENZSYN-16GS_c_FWD-YGR143W' 87.11394",
 "'ENZSYN-PRMICI_c_FWD-YIL020C' 31.48779",
 "'ENZSYN-P5CD_m_FWD-YHR037W' 65.93233",
 "'ENZSYN-P5CD_m_REV-YHR037W' 65.93233",
 "'ENZSYN-DHPPDA2_c_FWD-YOL066C' 69.34607",
 "'ENZSYN-DRAPPRy_c_FWD-YBR153W' 57.40262",
 "'ENZSYN-HPPK2_m_FWD-YNL256W' 190.488",
 "'ENZSYN-AATA_c_FWD-YJL060W' 100.96146",
 "'ENZSYN-AATA_c_REV-YJL060W' 100.96146",
 "'ENZSYN-AATA_c_FWD-YER152C' 98.08836",
 "'ENZSYN-AATA_c_REV-YER152C' 98.08836",
 "'ENZSYN-AATA_c_FWD-YGL202W' 114.5817",
 "'ENZSYN-AATA_c_REV-YGL202W' 114.5817",
 "'ENZSYN-DPR_c_FWD-YHR063C' 44.14436",
 "'ENZSYN-IPPMIb_c_FWD-YGL009C' 89.54808",
 "'ENZSYN-IPPMIb_c_REV-YGL009C' 89.54808",
 "'ENZSYN-IPPS_c_FWD-YOR108W' 139.5135",
 "'ENZSY